# Simulation studies in Python

This notebook illustrates techniques for conducting simulation studies in Python.
The main focus here is to better understand that properties of statistical
methods in various settings.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

nrep = 1000

# Simulation for assessing the standard error

One of the most basic facts in statistics is that the sample mean of independent
and identically distributed (iid) data has the form $\sigma/\sqrt{n}$, where
$n$ is the sample size, and $\sigma$ is the standard deviation.  We don't need
a simulation study to verify this provable fact, but we can use this as an
opportunity to conduct a simulation study in a setting where we know what the
answer is.

In [ ]:
# Sample sizes
for nobs in 50, 200, 800:

    # Standard deviation
    for sig in 0.5, 1, 2:

        # Population distributions
        for dist in 0, 1:

            # Simulate the data
            if dist == 0:
                x = sig * np.random.normal(size=(nrep, nobs))
            elif dist == 1:
                df = 4
                f = np.sqrt((df - 2) / df)
                x = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)

            # The standard error (obtained from the simulation)
            se = x.mean(1).std()

            # The standard error based on theory
            se_theory = sig / np.sqrt(nobs)

            print("%5d %12.4f %8.0f %12.4f %12.4f" % (nobs, sig, dist, se, se_theory))

A slightly more realistic example is to look at the standard error of the Pearson
correlation coefficient.  Unlike in the case of the sample mean for IID data, here we only
have an approximate analytic formula that the standard error is roughly $1/\sqrt{n}$.
The following simulation shows that when the population correlation is zero, this formula
works well when the true correlation is
small, for both normal and heavy-tailed data distributions:

In [ ]:
# The sample sizes
for nobs in 10, 50, 100:

    # The population distributions
    for dist in 0, 1:

        # Generate the data
        if dist == 0:
            x1 = sig * np.random.normal(size=(nrep, nobs))
            x2 = sig * np.random.normal(size=(nrep, nobs))
        elif dist == 1:
            df = 4
            f = np.sqrt((df - 2) / df)
            x1 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)
            x2 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)

        # The sample correlation coefficients
        cor = [np.corrcoef(x1[i, :], x2[i, :])[0, 1] for i in range(nrep)]
        cor = np.asarray(cor)

        # The standard error obtained via simulation
        se = cor.std()

        # The standard error based on theory
        se_theory = 1 / np.sqrt(nobs)

        print("%5d %8.0f %12.4f %12.4f" % (nobs, dist, se, se_theory))

Now we turn to settings where the true correlation is positive.  The Pearson
correlation coefficient has an inverse "mean/variance relationship" --
when the correlation is large, the standard deviation is small, and vice versa:

In [ ]:
# Sample sizes
for nobs in 10, 50, 100:

    # The population distributin
    for dist in 0, 1:

        # The population correlation
        for r in 0.25, 0.5, 0.75:

            # Generate the data
            if dist == 0:
                x1 = sig * np.random.normal(size=(nrep, nobs))
                x2 = sig * np.random.normal(size=(nrep, nobs))
            elif dist == 1:
                df = 4
                f = np.sqrt((df - 2) / df)
                x1 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)
                x2 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)

            # Induce the correlation
            x2 = r*x1 + np.sqrt(1 - r**2)*x2

            # Calculate the sample correlations
            cor = [np.corrcoef(x1[i, :], x2[i, :])[0, 1] for i in range(nrep)]
            cor = np.asarray(cor)

            # The standard error based on the simulation
            se = cor.std()

            # The (approxiate) standard deviation based on theory
            se_theory = 1 / np.sqrt(nobs)

            print("%5d %8.0f %12.4f %12.4f %12.4f" % (nobs, dist, r, se, se_theory))

A "variance-stabilizing transformation" is a transformation of the data that removes
a mean-variance relationship.  For normal data, the "Fisher Z-transform" (arc-tangent
function) is an approximate variance stabilizing transformation, but it is not
very effective for heavier-tailed data.

In [ ]:
# Sample sizes
for nobs in 10, 50, 100:

    # Population distribution (normal or t)
    for dist in 0, 1:

        # Population correlation
        for r in 0.25, 0.5, 0.75:

            if dist == 0:
                x1 = sig * np.random.normal(size=(nrep, nobs))
                x2 = sig * np.random.normal(size=(nrep, nobs))
            elif dist == 1:
                df = 4
                f = np.sqrt((df - 2) / df)
                x1 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)
                x2 = sig * f * np.random.standard_t(size=(nrep, nobs), df=df)

            # Induce the correlation
            x2 = r*x1 + np.sqrt(1 - r**2)*x2

            # Get the sample correlation
            cor = [np.corrcoef(x1[i, :], x2[i, :])[0, 1] for i in range(nrep)]
            cor = np.asarray(cor)

            # Apply the Fisher transformation
            tcor = 0.5 * np.log((1 + cor) / (1 - cor))

            # The simulation-based standard error
            se = tcor.std()

            # The theoretical SE
            se_theory = 1 / np.sqrt(nobs - 3)

            print("%5d %8.0f %12.4f %12.4f %12.4f" % (nobs, dist, r, se, se_theory))

As a final example of using simulation to assess standard errors, we can consider
$2 \times 2$ contingency tables.  Here we focus
on the log odds ratio, which is a commonly-used statistic for assessing the degree
of association between the rows and the columns of the table.

In [ ]:
# The expected total sample size
for n in 50, 100:

    # The population odds ratio between rows/columns
    for oddsratio in 1, 2:

        # Marginal probabilities of the first row
        for p1 in 0.1, 0.5:

            # Marginal probabilities of the second row
            for p2 in 0.1, 0.5:

                # Get the cell probabilities
                # https://en.wikipedia.org/wiki/Odds_ratio
                if oddsratio != 1:
                    s = np.sqrt((1 + (p1 + p2)*(oddsratio - 1))**2 + 4*oddsratio*(1-oddsratio)*p1*p2)
                    p11 = (1 + (p1 + p2) * (oddsratio - 1) - s) / (2 * (oddsratio - 1))
                else:
                    p11 = p1 * p2
                p12 = p1 - p11
                p21 = p2 - p11
                p22 = 1 - (p11 + p12 + p21)

                # Simulate the data for nrep 2x2 tables
                table = np.zeros((nrep, 4))
                table[:, 0] = np.random.poisson(n*p11, size=nrep)
                table[:, 1] = np.random.poisson(n*p12, size=nrep)
                table[:, 2] = np.random.poisson(n*p21, size=nrep)
                table[:, 3] = np.random.poisson(n*p22, size=nrep)

                # Get the sample log odds ratio (LOR) for every table
                sample_lor = np.log(table[:, 0]) + np.log(table[:, 3])
                sample_lor -= np.log(table[:, 1]) + np.log(table[:, 2])

                # Some LOR values are infinite due to zero cells, we drop
                # them here
                sample_lor = sample_lor[np.isfinite(sample_lor)]

                # The standard error of the LOR based on the simulation
                se = sample_lor.std()

                # The theoretical SE of the LOR
                se_theory = np.sqrt(1/p11 + 1/p12 + 1/p21 + 1/p22) / np.sqrt(n)

                print("%8d %12.4f %12.4f %12.4f %12.4f %12.4f" %
                      (n, p1, p2, oddsratio, se, se_theory))

# Sample size and power assessment

Simulation studies are commonly used for power analysis.  This is a big topic,
but broadly speaking, the goal of a power analysis is to assess what effect
sizes can be detected when applying a given dataset to a sample from a given
population.

Some statistical methods are amenable to analytic-based power analyses.  But in many
cases it is more practical to use simulation.  Below is an example of a simulation-based
power analysis for logistic regression.

In [ ]:

# The sample size
for n in 100, 200:

    # The correlation between two covariates (multicollinearity)
    for r in 0, 0.5:

        # The effect size
        for b in 0.1, 0.5:

            power = 0
            for k in range(100):

                # Generate covariates
                x = np.random.normal(size=(n, 3))
                x[:, 0] = 1
                x[:, 2] = r*x[:, 1] + np.sqrt(1 - r**2)*x[:, 2]

                # The linear predictor
                lpr = b * x[:, 1]

                # The outcome probabilities
                pr = 1 / (1 + np.exp(-lpr))

                # The outcomes
                y = (np.random.uniform(size=n) < pr).astype(np.int)

                # Fit the model, and see if we detect the effect of interest
                model = sm.GLM(y, x, family=sm.families.Binomial())
                result = model.fit()
                power += np.abs(result.tvalues[1]) > 2

            power /= 100
            print("%8d %12.4f %12.4f %12.4f" % (n, r, b, power))

Simulation studies can also be useful for assessing bias in non-standard
settings.  For example, below we consider omitted variable bias in logistic
regression.

In [ ]:

b = 0.5

# The sample size
for n in 100, 200:

    # The correlation between two covariates (multicollinearity)
    for r in 0, 0.5, 0.8:

        est = 0
        for k in range(500):

            # Generate covariates
            x = np.random.normal(size=(n, 3))
            x[:, 0] = 1
            x[:, 2] = r*x[:, 1] + np.sqrt(1 - r**2)*x[:, 2]

            # The linear predictor
            lpr = b * (x[:, 1] + x[:, 2])

            # The outcome probabilities
            pr = 1 / (1 + np.exp(-lpr))

            # The outcomes
            y = (np.random.uniform(size=n) < pr).astype(np.int)

            # Fit the model omitting one variable
            model = sm.GLM(y, x[:, 0:2], family=sm.families.Binomial())
            result = model.fit()
            est += result.params[1]

        est /= 500
        print("%8d %12.4f %12.4f %12.4f" % (n, r, b, est))

Below is another example of a simulation study used to assess bias.  Here
the bias is a result of "informative" missing data, or selection bias.

In [ ]:

b = 0.5

# The sample size
for n in 100, 200:

    # The correlation between two covariates (multicollinearity)
    for r in 0, 0.5, 0.8:

        est = 0
        for k in range(500):

            # Generate covariates
            x = np.random.normal(size=(n, 3))
            x[:, 0] = 1
            x[:, 2] = r*x[:, 1] + np.sqrt(1 - r**2)*x[:, 2]

            # The linear predictor
            lpr = b * (x[:, 1] + x[:, 2])

            # The outcome probabilities
            pr = 1 / (1 + np.exp(-lpr))

            # The outcomes
            y = (np.random.uniform(size=n) < pr).astype(np.int)

            ii = np.flatnonzero(x[:,1] * (2*y-1) + 2*np.random.normal(size=n) > 0)
            y = y[ii]
            x = x[ii, :]

            # Fit the model omitting one variable
            model = sm.GLM(y, x[:, 0:2], family=sm.families.Binomial())
            result = model.fit()
            est += result.params[1]

        est /= 500
        print("%8d %12.4f %12.4f %12.4f" % (n, r, b, est))